In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

#### Char-RNN to generate text

In [6]:
# Sample text
text = "hello"

# Step 1: Preprocess - Create vocabulary
chars = sorted(list(set(text)))
char_to_idx = {ch: i for i, ch in enumerate(chars)}
idx_to_char = {i: ch for ch, i in char_to_idx.items()}
vocab_size = len(chars)

# Encode text to integer indices
encoded_text = [char_to_idx[c] for c in text]

# Step 2: Define the CharRNN model
class CharRNN(nn.Module):
    def __init__(self, vocab_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, hidden_size)         # Character Embeddings
        self.rnn = nn.RNN(hidden_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size)               # Output layer to vocab size

    def forward(self, x, hidden=None):
        x = self.embed(x)              # Convert character indices to embeddings
        output, hidden = self.rnn(x, hidden)  # Process sequence with RNN
        logits = self.fc(output)       # Map to output vocabulary
        return logits, hidden

# Step 3: Instantiate model
hidden_size = 16
model = CharRNN(vocab_size, hidden_size)

# Step 4: Prepare training data (1-step toy training)
input_seq = torch.tensor([encoded_text[:-1]])   # "hell"
target_seq = torch.tensor([encoded_text[1:]])   # "ello"

# Step 5: Forward pass
logits, _ = model(input_seq)
loss = F.cross_entropy(logits.view(-1, vocab_size), target_seq.view(-1))
loss.backward()

#### Role of charcter enbedding and softmax output layer

In [7]:
# Step 6: Predict next character
last_logits = logits[0, -1]              # Output of last time step
prob_dist = F.softmax(last_logits, dim=0)

next_char_idx = torch.multinomial(prob_dist, num_samples=1).item()
next_char = idx_to_char[next_char_idx]

print("Predicted next char:", next_char)

Predicted next char: e


#### Temperature sampling influence

In [8]:
# Step 7: Sampling function with temperature
def sample_with_temperature(logits, temperature=1.0):
    logits = logits / temperature
    probs = F.softmax(logits, dim=0)
    return torch.multinomial(probs, num_samples=1).item()

# Step 8: Test different temperatures
for temp in [0.5, 1.0, 1.5]:
    idx = sample_with_temperature(last_logits, temperature=temp)
    print(f"Temp {temp}: {idx_to_char[idx]}")

Temp 0.5: e
Temp 1.0: l
Temp 1.5: o
